In [1]:
# ====================================================================
# CÉLULA 1: COLETA DE DADOS GEOESPACIAIS (POIs) DO OSM - VERSÃO CORRIGIDA
# ====================================================================

import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pathlib import Path
import time
import warnings

# --- 1. CONFIGURAÇÕES ---
print(" Iniciando a coleta de dados do OpenStreetMap...")

DATA_DIR = Path("../data/geoespacial")
DATA_DIR.mkdir(parents=True, exist_ok=True)

LIMITE_FEATURES_POR_POI = 250  # Aumentado para capturar mais dados para agregação
BUFFER_RADIUS_M = 5000
PAUSA_ENTRE_REQUISICOES_S = 1.5

locais_turisticos = [
    {"nome": "Quedas de Calandula", "lat": -9.075025, "lon": 16.001131, "provincia": "Malanje"},
    {"nome": "Miradouro da Lua", "lat": -9.221147, "lon": 13.090001, "provincia": "Luanda"},
    {"nome": "Museu Kulumbimbi", "lat": -6.264389, "lon": 14.245581, "provincia": "Zaire"},
    {"nome": "Reserva Parcial do Namibe", "lat": -15.766760, "lon": 12.399914, "provincia": "Namibe"},
    {"nome": "Fortaleza de São Miguel", "lat": -8.808343, "lon": 13.223444, "provincia": "Luanda"},
    {"nome": "Ilha do Mussulo", "lat": -8.964635, "lon": 13.050462, "provincia": "Luanda"},
    {"nome": "Serra da Leba", "lat": -15.071753, "lon": 13.236296, "provincia": "Huila"},
    {"nome": "Cristo Rei Lubango", "lat": -14.940039, "lon": 13.511860, "provincia": "Huíla"},
    {"nome": "Serra da Leba", "lat": -15.0788, "lon": 13.2397, "provincia": "Huíla"},
    {"nome": "Fenda da Tundavala", "lat": -14.817465, "lon": 13.381539, "provincia": "Huíla"},
    {"nome": "Baía Azul", "lat": -12.627978, "lon": 13.234185, "provincia": "Benguela"},
    {"nome": "Quedas do Rio Chiumbe", "lat": -11.021380, "lon": 20.203180, "provincia": "Moxico"},
    {"nome": "Parque Nacional de Cameia", "lat": -11.883338, "lon": 21.666681, "provincia": "Moxico"},
    {"nome": "Grutas do Nzenzo", "lat": -7.520192, "lon": 14.565103, "provincia": "Uíge"}
]

TAGS_OSM = {
    "tourism": True, "amenity": True, "leisure": True, "natural": True,
    "landuse": True, "highway": True, "waterway": True, "power": True,
    "man_made": True, "railway": True, "building": True
}

# --- 2. FUNÇÃO DE COLETA (sem alterações) ---

def coletar_features_osm(lat, lon, tags, buffer_m):
    """Retorna um GeoDataFrame com as features do OSM dentro de um buffer."""
    try:
        ponto_central = gpd.GeoDataFrame([["ponto", Point(lon, lat)]], columns=["nome", "geometry"], crs="EPSG:4326")
        area_busca = ponto_central.to_crs(3857).buffer(buffer_m).to_crs(4326).iloc[0]
        gdf_features = ox.features_from_polygon(area_busca, tags).reset_index()
        return gdf_features
    except Exception as e:
        print(f"    ↳   Aviso durante a coleta: {e}")
        return None

# --- 3. LOOP DE EXECUÇÃO ---

lista_gdfs_coletados = []

for local in locais_turisticos:
    print(f"\n Processando: {local['nome']} ({local['provincia']})...")

    features = coletar_features_osm(local['lat'], local['lon'], TAGS_OSM, BUFFER_RADIUS_M)

    if features is not None and not features.empty:
        total_encontrado = len(features)
        print(f"    ↳ {total_encontrado} features encontradas.")

        if total_encontrado > LIMITE_FEATURES_POR_POI:
            features = features.sample(n=LIMITE_FEATURES_POR_POI, random_state=42)
            print(f"    ↳ Reduzido para uma amostra aleatória de {len(features)} features.")
        
        # --- ENRIQUECIMENTO E PADRONIZAÇÃO ---
        features['ponto_turistico'] = local['nome']
        features['provincia'] = local['provincia']
        
        # ### ALTERAÇÃO PRINCIPAL ###
        # Adiciona as coordenadas do PONTO TURÍSTICO CENTRAL a cada linha.
        # Isso é essencial para a agregação posterior.
        features['poi_lat'] = local['lat']
        features['poi_lon'] = local['lon']
        
        # Calcula a lat/lon de cada feature individual (centroide)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            centroids = features.to_crs(3857).geometry.centroid.to_crs(4326)
            features['feature_lat'] = centroids.y
            features['feature_lon'] = centroids.x
        
        lista_gdfs_coletados.append(features)
    else:
        print(f"    ↳ Nenhuma feature encontrada.")
        
    time.sleep(PAUSA_ENTRE_REQUISICOES_S)

# --- 4. COMBINAÇÃO FINAL E SALVAMENTO ---

if lista_gdfs_coletados:
    print("\n🔹 Combinando todos os dados coletados...")
    gdf_final_raw = gpd.GeoDataFrame(pd.concat(lista_gdfs_coletados, ignore_index=True), crs="EPSG:4326")
    
    # Renomear 'name' para 'feature_name' para clareza
    if 'name' in gdf_final_raw.columns:
        gdf_final_raw = gdf_final_raw.rename(columns={'name': 'feature_name'})
    
    output_path = DATA_DIR / "pontos_turisticos_angola_raw.geojson"
    gdf_final_raw.to_file(output_path, driver="GeoJSON")
    
    print(f"\n Processo concluído com sucesso!")
    print(f"   Arquivo bruto salvo em: {output_path}")
    print(f"   Total de features coletadas: {len(gdf_final_raw)}")

    print("\n--- Verificação da Estrutura do DataFrame Final ---")
    print("Colunas presentes no arquivo final (amostra):")
    # ### ALTERAÇÃO NA VERIFICAÇÃO ###
    # Mostra as colunas novas e mais importantes
    cols_to_check = [
        'ponto_turistico', 'provincia', 'poi_lat', 'poi_lon', 
        'feature_name', 'feature_lat', 'feature_lon', 'tourism', 'amenity'
    ]
    existing_cols_to_check = [c for c in cols_to_check if c in gdf_final_raw.columns]
    print(gdf_final_raw[existing_cols_to_check].head())
else:
    print("\n Nenhuma feature foi coletada em nenhuma das localidades.")
 

 Iniciando a coleta de dados do OpenStreetMap...

 Processando: Quedas de Calandula (Malanje)...


    ↳ 79 features encontradas.



 Processando: Miradouro da Lua (Luanda)...


    ↳ 159 features encontradas.



 Processando: Museu Kulumbimbi (Zaire)...


...


 Processando: Parque Nacional de Cameia (Moxico)...


    ↳ 1 features encontradas.



 Processando: Grutas do Nzenzo (Uíge)...
    ↳ 17 features encontradas.



🔹 Combinando todos os dados coletados...



 Processo concluído com sucesso!
   Arquivo bruto salvo em: ..\data\geoespacial\pontos_turisticos_angola_raw.geojson
   Total de features coletadas: 1719

--- Verificação da Estrutura do DataFrame Final ---
Colunas presentes no arquivo final (amostra):
       ponto_turistico provincia   poi_lat    poi_lon          feature_name  \
0  Quedas de Calandula   Malanje -9.075025  16.001131   Quedas de Calandula   
1  Quedas de Calandula   Malanje -9.075025  16.001131                   NaN   
2  Quedas de Calandula   Malanje -9.075025  16.001131                   NaN   
3  Quedas de Calandula   Malanje -9.075025  16.001131                   NaN   
4  Quedas de Calandula   Malanje -9.075025  16.001131  Estrada de Calandula   

   feature_lat  feature_lon    tourism amenity  
0    -9.074067    16.000316        NaN     NaN  
1    -9.074009    15.998896        NaN     NaN  
2    -9.074206    16.002487        NaN     NaN  
3    -9.077999    16.002056  camp_site     NaN  
4    -9.111147    15.97091

In [2]:
# ====================================================================
# CÉLULA 2: NORMALIZAÇÃO, ENRIQUECIMENTO E ENGENHARIA ESPACIAL (COMBINADA)
# ====================================================================
import geopandas as gpd
import pandas as pd
from pathlib import Path
from scipy.spatial import cKDTree
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# --- 1. CONFIGURAÇÃO E CARREGAMENTO ---
print(" Iniciando o processamento completo dos dados geoespaciais...")
DATA_DIR = Path("../data/geoespacial")
input_path = DATA_DIR / "pontos_turisticos_angola_raw.geojson"
output_path = DATA_DIR / "pontos_turisticos_angola_normalizado.geojson"

try:
    gdf = gpd.read_file(input_path)
    print(f" Arquivo bruto '{input_path.name}' carregado com {len(gdf)} features.")
except Exception as e:
    # Usar 'raise' interrompe a execução se o arquivo base não for encontrado, o que é o comportamento desejado.
    raise RuntimeError(f"❌ ERRO CRÍTICO: Não foi possível carregar o arquivo de entrada: {e}")

# --- 2. LIMPEZA E DESDUPLICAÇÃO ---
print(" Realizando limpeza...")
gdf = gdf[gdf.geometry.notna() & gdf.is_valid]
if 'feature_name' in gdf.columns:
    gdf = gdf.drop_duplicates(subset=['feature_name', 'geometry'], keep='first')
print(f"  {len(gdf)} features restantes após limpeza.")

# --- 3. NORMALIZAÇÃO DE CATEGORIAS ---
print("Normalizando categorias de 'tourism' e 'amenity'...")

# Dicionários de mapeamento para agrupar categorias
map_tourism = {
    "guest_house": "alojamento", "hotel": "alojamento", "hostel": "alojamento",
    "motel": "alojamento", "apartment": "alojamento", "chalet": "alojamento",
    "museum": "cultura", "artwork": "cultura",
    "attraction": "atracao", "viewpoint": "miradouro",
    "gallery": "arte", "zoo": "lazer"
}
map_amenity = {
    "restaurant": "restauracao", "fast_food": "restauracao", "cafe": "restauracao",
    "bar": "restauracao", "food_court": "restauracao",
    "hospital": "saude", "clinic": "saude", "doctors": "saude", "dentist": "saude",
    "school": "educacao", "university": "educacao", "college": "educacao",
    "bank": "financeiro", "atm": "financeiro", "bureau_de_change": "financeiro",
    "parking": "infraestrutura", "bus_station": "infraestrutura", "taxi": "infraestrutura",
    "ferry_terminal": "infraestrutura"
}

# Função segura para aplicar o mapeamento
def normalizar_coluna(df, coluna, mapa):
    if coluna in df.columns:
        # Assegura que a coluna é do tipo string para usar o .replace
        df[coluna] = df[coluna].astype(str).replace(mapa)
    return df

gdf = normalizar_coluna(gdf, 'tourism', map_tourism)
gdf = normalizar_coluna(gdf, 'amenity', map_amenity)
print(" Categorias normalizadas.")


# --- 4. ENGENHARIA DE FEATURES ESPACIAIS ---
print(" Iniciando engenharia de features espaciais (cálculos em metros)...")
CRS_METRIC = "EPSG:32733" # UTM Zone 33S para Angola
gdf_metric = gdf.to_crs(CRS_METRIC)

# 4.1. Distância ao vizinho mais próximo
print("   - Calculando distância à feature mais próxima...")
centroids = gdf_metric.geometry.centroid
coords = np.array(list(zip(centroids.x, centroids.y)))
kdtree = cKDTree(coords)
# k=2 porque o vizinho mais próximo (k=1) é o próprio ponto
distances, _ = kdtree.query(coords, k=2, workers=-1) # workers=-1 usa todos os cores
gdf_metric['dist_nearest_feature_m'] = distances[:, 1]

# 4.2. Contagem de features próximas (Densidade)
print("   - Contando features em um raio de 1km...")
BUFFER_RADIUS_M = 1000
# sjoin é uma forma muito mais rápida de fazer essa contagem
buffers = gdf_metric.geometry.buffer(BUFFER_RADIUS_M)
# Contar quantos pontos caem dentro de cada buffer
# O resultado de sjoin_counts tem um índice alinhado com gdf_metric
join = gpd.sjoin(gpd.GeoDataFrame(geometry=buffers), gdf_metric, how='left', predicate='contains')
counts = join.groupby(join.index).size() - 1 # Subtrai 1 para não contar o próprio ponto
gdf_metric['features_within_1km'] = counts.reindex(gdf_metric.index).fillna(0)

print("    Variáveis espaciais calculadas.")


# --- 5. FINALIZAÇÃO E SALVAMENTO ---
print(" Finalizando e salvando o GeoDataFrame enriquecido...")

# Voltar para a projeção original (lat/lon)
gdf_final = gdf_metric.to_crs(gdf.crs)

# Neste ponto, mantemos todas as colunas. A seleção final será feita na agregação.
gdf_final.to_file(output_path, driver="GeoJSON", encoding='utf-8')

print(f"   Processo concluído! Arquivo normalizado e enriquecido salvo em: {output_path}")
print(f"   O arquivo final tem {gdf_final.shape[0]} linhas e {gdf_final.shape[1]} colunas.")
print("\n--- Amostra do Dataset Enriquecido ---")
cols_to_show = [
    'ponto_turistico', 'provincia', 'feature_name', 'tourism', 'amenity',
    'dist_nearest_feature_m', 'features_within_1km'
]
existing_cols_to_show = [c for c in cols_to_show if c in gdf_final.columns]
display(gdf_final[existing_cols_to_show].head())

 Iniciando o processamento completo dos dados geoespaciais...


 Arquivo bruto 'pontos_turisticos_angola_raw.geojson' carregado com 1719 features.
 Realizando limpeza...
  1676 features restantes após limpeza.
Normalizando categorias de 'tourism' e 'amenity'...
 Categorias normalizadas.
 Iniciando engenharia de features espaciais (cálculos em metros)...
   - Calculando distância à feature mais próxima...
   - Contando features em um raio de 1km...


    Variáveis espaciais calculadas.
 Finalizando e salvando o GeoDataFrame enriquecido...


   Processo concluído! Arquivo normalizado e enriquecido salvo em: ..\data\geoespacial\pontos_turisticos_angola_normalizado.geojson
   O arquivo final tem 1676 linhas e 528 colunas.

--- Amostra do Dataset Enriquecido ---


,ponto_turistico,provincia,feature_name,tourism,amenity,dist_nearest_feature_m,features_within_1km
0,Quedas de Calandula,Malanje,Quedas de Calandula,None,None,83.325233,28
1,Quedas de Calandula,Malanje,None,None,None,48.478089,28
2,Quedas de Calandula,Malanje,None,None,None,113.281622,29
3,Quedas de Calandula,Malanje,None,camp_site,None,27.721497,29
4,Quedas de Calandula,Malanje,Estrada de Calandula,None,None,2375.771517,5


In [3]:
# ==============================================================
# NORMALIZAÇÃO DOS DADOS DE MOBILIDADE E DE INFRAESTRUTURA
# ==============================================================

import pandas as pd
import geopandas as gpd
from pathlib import Path

# === Diretório e arquivo ===
DATA_DIR = Path("../data/climatic-environmental")
DATA_DIR.mkdir(parents=True, exist_ok=True)
input_path = DATA_DIR / "mobilidade_infra.csv"

# === Carregar ===
df = pd.read_csv(input_path)

# === Normalizar nomes de colunas ===
df.columns = df.columns.str.strip().str.lower().str.replace(r'\s+', '_', regex=True)

# === Preenchimento básico ===
num_cols = df.select_dtypes(include=["number"]).columns.tolist()
for c in num_cols:
    df[c] = df[c].fillna(df[c].median())

cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
for c in cat_cols:
    df[c] = df[c].fillna("desconhecido")

# === Limites plausíveis ===
if "velocidade_media_kmh" in df.columns:
    df["velocidade_media_kmh"] = df["velocidade_media_kmh"].clip(lower=0, upper=200)

if "capacidade" in df.columns:
    df["capacidade"] = df["capacidade"].clip(lower=0)

if "fluxo_diario" in df.columns:
    df["fluxo_diario"] = df["fluxo_diario"].clip(lower=0)

# === Datas ===
for date_col in ["date", "data", "timestamp"]:
    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

# === Normalizar categorias de infraestrutura ===
infra_col_candidates = [c for c in df.columns if c in ("tipo", "tipo_infra", "infra_type", "category")]
if infra_col_candidates:
    infra_col = infra_col_candidates[0]
    df[infra_col] = df[infra_col].astype(str).str.lower()
    df["is_road"] = df[infra_col].str.contains(r"road|highway|estrada", na=False).astype(int)
    df["is_rail"] = df[infra_col].str.contains(r"rail|ferrovia", na=False).astype(int)
    df["is_port"] = df[infra_col].str.contains(r"port|porto", na=False).astype(int)
    df["is_airport"] = df[infra_col].str.contains(r"airport|aeroporto", na=False).astype(int)

# === Coordenadas -> GeoDataFrame ===
lat_candidates = [c for c in df.columns if c in ("latitude", "lat", "y")]
lon_candidates = [c for c in df.columns if c in ("longitude", "lon", "lng", "x")]

output_csv = DATA_DIR / "mobilidade_infra_normalizado.csv"
output_geo = DATA_DIR / "mobilidade_infra_normalizado.geojson"

if lat_candidates and lon_candidates:
    lat_col = lat_candidates[0]
    lon_col = lon_candidates[0]
    df = df.dropna(subset=[lat_col, lon_col])
    gdf = gpd.GeoDataFrame(
        df.copy(),
        geometry=gpd.points_from_xy(df[lon_col].astype(float), df[lat_col].astype(float)),
        crs="EPSG:4326"
    )
    gdf.to_file(output_geo, driver="GeoJSON")
    df.to_csv(output_csv, index=False)
    print(f"Dados de mobilidade normalizados salvos em:\n   - {output_csv}\n   - {output_geo}")
else:
    df.to_csv(output_csv, index=False)
    print(f"Dados de mobilidade normalizados salvos em: {output_csv}")
    print(df.head())



Dados de mobilidade normalizados salvos em: ..\data\climatic-environmental\mobilidade_infra_normalizado.csv
                  nome província  distância_estrada_principal_(km)  \
0            Baía Azul  Benguela                               1.2   
1           Cristo Rei  Benguela                               0.8   
2  Quedas de Kalandula   Malanje                               6.5   
3        Serra da Leba     Huíla                               1.0   
4     Miradouro da Lua    Luanda                               2.5   

   distância_cidade_(km) acessibilidade tipo_via_acesso infraestrutura  \
0                    7.5            Boa     Pavimentada       Completa   
1                    5.0            Boa     Pavimentada       Completa   
2                   75.0          Média    Terra batida         Básica   
3                   15.0            Boa     Pavimentada          Média   
4                   45.0            Boa     Pavimentada       Completa   

           serviços_dispon

In [4]:
import pandas as pd
from pathlib import Path

# ==============================================================
# NORMALIZAÇÃO DOS DADOS CLIMATICOS-AMBIENTAIS

# ==============================================================
# === Diretório e arquivos ===
DATA_DIR = Path("../data/climatic-environmental")
DATA_DIR.mkdir(parents=True, exist_ok=True)

input_clima = DATA_DIR / "1clima_lulc.csv"
df_clima = pd.read_csv(input_clima)

# === Normalização dos dados ===
# Temperatura média anual (°C)
df_clima['temp_med_anual'] = df_clima['temp_med_anual'].clip(20, 35)

# Precipitação anual (mm)
df_clima['precipitacao_anual'] = df_clima['precipitacao_anual'].clip(0, 2000)

# Índices de vegetação e água
for col in ['NDVI', 'EVI', 'NDWI']:
    df_clima[col] = df_clima[col].clip(-1, 1)

# Códigos LULC (Land Use Land Cover)
df_clima['LULC_codigo'] = df_clima['LULC_codigo'].astype('category')

# Altitude (m)
df_clima['altitude'] = df_clima['altitude'].clip(0, 2500)

# === Salvar dados normalizados ===
output_path = DATA_DIR / "clima_lulc_normalizado.csv"
df_clima.to_csv(output_path, index=False)
print(f" Dados climáticos-ambientais normalizados salvos em: {output_path}")
print(df_clima.head())

 Dados climáticos-ambientais normalizados salvos em: ..\data\climatic-environmental\clima_lulc_normalizado.csv
  nome_ponto_turistico provincia  temp_med_anual  precipitacao_anual  NDVI  \
0  Quedas de Calandula   Malanje            26.3                1290  0.72   
1     Miradouro da Lua    Luanda            27.8                 320  0.26   
2    Parque da Kissama    Luanda            26.9                 310  0.33   
3            Baia Azul  Benguela            27.1                 280  0.05   
4    Reserva do Namibe    Namibe            28.5                  90  0.02   

    EVI  NDWI LULC_codigo  altitude  
0  0.64  0.09          20      1058  
1  0.18  0.12          60       112  
2  0.25  0.07          30       180  
3  0.11  0.04          60        12  
4  0.08  0.01          60        60  


In [5]:
import pandas as pd
from pathlib import Path

# ==============================================================
# NORMALIZAÇÃO DOS DADOS ECONÔMICOS-SOCIAIS
# ==============================================================
# === Diretório e arquivos ===
DATA_DIR = Path("../data/economicsocial")
DATA_DIR.mkdir(parents=True, exist_ok=True)

input_eco = DATA_DIR / "dados_socioeconomicos.csv"
df_eco = pd.read_csv(input_eco)

# === Normalização dos dados ===
# PIB per capita 
df_eco['pib_per_capita'] = df_eco['pib_per_capita'].clip(500, 5000)

# População
df_eco['populacao'] = df_eco['populacao'].clip(1000, 1000000)

# Índice de desenvolvimento humano
df_eco['idh'] = df_eco['idh'].clip(0, 1)

# Densidade populacional (hab/km²)
df_eco['densidade_pop'] = df_eco['densidade_pop'].clip(0, 1000)

# Taxa de urbanização (%)
df_eco['taxa_urbanizacao'] = df_eco['taxa_urbanizacao'].clip(0, 100)

# === Salvar dados normalizados ===
output_path = DATA_DIR / "economicsocial_normalizado.csv"
df_eco.to_csv(output_path, index=False)
print(f" Dados econômicos-sociais normalizados salvos em: {output_path}")
print(df_eco.head())

 Dados econômicos-sociais normalizados salvos em: ..\data\economicsocial\economicsocial_normalizado.csv
                        nome provincia  populacao  densidade_pop  \
0        Quedas de Calandula   Malanje     129958           77.1   
1           Miradouro da Lua    Luanda     154867          298.1   
2           Museu Kulumbimbi     Zaire     139932           39.5   
3  Reserva Parcial do Namibe    Namibe     111694          329.1   
4    Fortaleza de São Miguel    Luanda     127879          423.3   

   pib_per_capita    idh  taxa_urbanizacao  emprego_turismo  
0         1898.31  0.522              80.6              3.2  
1         1471.82  0.655              68.6              8.1  
2         3055.16  0.633              44.6              7.7  
3         2347.39  0.700              59.2              6.5  
4         4149.69  0.502              56.3              3.2  


In [6]:
# ====================================================================
# SCRIPT DE GERAÇÃO DE VARIÁVEIS ESPACIAIS - VERSÃO OTIMIZADA
# ====================================================================

import geopandas as gpd
from pathlib import Path
from scipy.spatial import cKDTree
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# --- 1. CONFIGURAÇÃO DE CAMINHOS ---
DATA_DIR = Path("../data/geoespacial")
# Usamos o _raw como entrada, pois ele contém todas as features
input_path = DATA_DIR / "pontos_turisticos_angola_raw.geojson" 
# A saída será o _normalizado, agora enriquecido com variáveis espaciais
output_path = DATA_DIR / "pontos_turisticos_angola_normalizado.geojson" 

print("Carregando dados brutos para enriquecimento...")
try:
    gdf = gpd.read_file(input_path)
    print(f" Arquivo '{input_path.name}' carregado com {len(gdf)} features.")
except Exception as e:
    print(f"🚨 ERRO CRÍTICO: Não foi possível carregar o arquivo de entrada: {e}")
    exit()

# --- 2. PREPARAÇÃO DO GEODATAFRAME ---
# Converter para uma projeção métrica (em metros) para cálculos precisos
CRS_METRIC = "EPSG:32733" # UTM Zone 33S para Angola
gdf_metric = gdf.to_crs(CRS_METRIC)
print(f"Projeção convertida para {CRS_METRIC} para cálculos em metros.")


# --- 3. CÁLCULO DE DISTÂNCIA AO VIZINHO MAIS PRÓXIMO  ---
print("Calculando a distância ao vizinho mais próximo...")

# Usar o centroide garante que funcione para pontos, linhas e polígonos
centroids = gdf_metric.geometry.centroid
coords = np.array(list(zip(centroids.x, centroids.y)))

# Construir a árvore espacial para busca rápida
kdtree = cKDTree(coords)

# Encontrar o segundo vizinho mais próximo (o primeiro é o próprio ponto)
distances, _ = kdtree.query(coords, k=2)
nearest_distances = distances[:, 1]

# Adicionar a nova coluna ao GeoDataFrame
gdf_metric['dist_nearest_feature_m'] = nearest_distances


# --- 4. CONTAGEM DE FEATURES PRÓXIMAS (DENSIDADE - OTIMIZADO) ---
print("Contando features dentro de um raio de 1km...")
BUFFER_RADIUS_M = 1000

# Usar o índice espacial para uma contagem eficiente
spatial_index = gdf_metric.sindex
counts = []
for i, row in gdf_metric.iterrows():
    buffer = row.geometry.buffer(BUFFER_RADIUS_M)
    possible_matches_index = list(spatial_index.intersection(buffer.bounds))
    precise_matches = gdf_metric.iloc[possible_matches_index]
    points_within = precise_matches[precise_matches.geometry.within(buffer)]
    counts.append(len(points_within) - 1)

gdf_metric['features_within_1km'] = counts


# --- 5. FLAG DE PROXIMIDADE A CATEGORIAS (OTIMIZADO) ---
print("Verificando a proximidade a categorias específicas...")

# Filtrar para criar GeoDataFrames para cada categoria de interesse
hospitais = gdf_metric[gdf_metric['amenity'] == 'hospital'].copy()
escolas = gdf_metric[gdf_metric['amenity'] == 'school'].copy()
restaurantes = gdf_metric[gdf_metric['amenity'] == 'restaurant'].copy()

# Função otimizada para criar a flag de proximidade
def criar_flag_proximidade(gdf_base, gdf_alvo, nome_coluna, raio_m=1000):
    if not gdf_alvo.empty:
        join_prox = gpd.sjoin_nearest(gdf_base, gdf_alvo, distance_col="distancia")
        distancias_df = join_prox[['distancia']]
        distancias_df = distancias_df[~distancias_df.index.duplicated(keep='first')]
        flag_series = (distancias_df['distancia'] <= raio_m)
        gdf_base[nome_coluna] = flag_series.reindex(gdf_base.index).fillna(False).astype(int)
    else:
        gdf_base[nome_coluna] = 0
    return gdf_base

# Aplicar a função para cada categoria
gdf_metric = criar_flag_proximidade(gdf_metric, hospitais, 'hospital_nearby_1km')
gdf_metric = criar_flag_proximidade(gdf_metric, escolas, 'school_nearby_1km')
gdf_metric = criar_flag_proximidade(gdf_metric, restaurantes, 'restaurant_nearby_1km')


# --- 6. FINALIZAÇÃO E SALVAMENTO ---
print("Finalizando e salvando o resultado...")

# Voltar para a projeção original (lat/lon)
gdf_final = gdf_metric.to_crs(gdf.crs)

# Selecionar um conjunto final de colunas para manter o arquivo limpo
colunas_finais = [
    'ponto_turistico', 'provincia', 'tourism', 'amenity', 
    'leisure', 'natural', 'latitude', 'longitude', 
    'dist_nearest_feature_m', 'features_within_1km',
    'hospital_nearby_1km', 'school_nearby_1km', 'restaurant_nearby_1km',
    'geometry'
]
colunas_existentes = [col for col in colunas_finais if col in gdf_final.columns]
gdf_final_limpo = gdf_final[colunas_existentes]


# Salvar no arquivo de saída
gdf_final_limpo.to_file(output_path, driver="GeoJSON")

print(f"\n Processo de geração de variáveis espaciais concluído!")
print(f" Arquivo enriquecido salvo em: {output_path}")
print("\n--- Amostra do Dataset Enriquecido ---")
display(gdf_final_limpo.head())

Carregando dados brutos para enriquecimento...


 Arquivo 'pontos_turisticos_angola_raw.geojson' carregado com 1719 features.
Projeção convertida para EPSG:32733 para cálculos em metros.
Calculando a distância ao vizinho mais próximo...
Contando features dentro de um raio de 1km...


Verificando a proximidade a categorias específicas...
Finalizando e salvando o resultado...



 Processo de geração de variáveis espaciais concluído!
 Arquivo enriquecido salvo em: ..\data\geoespacial\pontos_turisticos_angola_normalizado.geojson

--- Amostra do Dataset Enriquecido ---


,ponto_turistico,provincia,tourism,amenity,leisure,natural,latitude,longitude,dist_nearest_feature_m,features_within_1km,hospital_nearby_1km,school_nearby_1km,restaurant_nearby_1km,geometry
0,Quedas de Calandula,Malanje,None,None,None,None,None,None,83.325233,28,0,0,1,POINT (16.00032 -9.07407)
1,Quedas de Calandula,Malanje,None,None,None,None,None,None,48.478089,28,0,0,1,POINT (15.9989 -9.07401)
2,Quedas de Calandula,Malanje,None,None,None,None,None,None,113.281622,29,0,0,1,POINT (16.00249 -9.07421)
3,Quedas de Calandula,Malanje,camp_site,None,None,None,None,None,27.721497,29,0,0,1,POINT (16.00206 -9.078)
4,Quedas de Calandula,Malanje,None,None,None,None,None,None,2375.771517,5,0,0,0,"LINESTRING (15.97131 -9.12829, 15.97187 -9.127..."


In [7]:
# ====================================================================
# CÉLULA FINAL: INTEGRAÇÃO E AGREGAÇÃO PARA GERAR model_input.csv
# ====================================================================

import pandas as pd
import geopandas as gpd
from pathlib import Path
import re
import warnings

warnings.filterwarnings('ignore')

print(" Iniciando a integração final de todas as fontes de dados...")

# --- 1. FUNÇÕES AUXILIARES (sem alterações) ---

def normalizar_texto(texto):
    """Função robusta para limpar e padronizar textos para junção."""
    if pd.isna(texto): return ""
    texto = str(texto).strip().lower()
    texto = re.sub(r"[áàãâä]", "a", texto); texto = re.sub(r"[éèêë]", "e", texto)
    texto = re.sub(r"[íìîï]", "i", texto); texto = re.sub(r"[óòõôö]", "o", texto)
    texto = re.sub(r"[úùûü]", "u", texto); texto = re.sub(r"ç", "c", texto)
    texto = re.sub(r"[^a-z0-9_]", "", texto) # Permite underscore
    return texto

def carregar_e_preparar(caminho, chaves_possiveis, nome_df="", is_geo=False):
    """
    Função unificada para carregar, normalizar colunas e criar a chave de junção.
    """
    try:
        df = gpd.read_file(caminho) if is_geo else pd.read_csv(caminho, encoding='utf-8')
        df.columns = [normalizar_texto(c) for c in df.columns]
        coluna_chave = next((col for col in chaves_possiveis if col in df.columns), None)
        
        if coluna_chave:
            df['key'] = df[coluna_chave].astype(str).apply(lambda x: normalizar_texto(x.replace('_', '')))
            print(f" {nome_df}: Arquivo carregado. Chave criada a partir de '{coluna_chave}'.")
            return df
        else:
            warnings.warn(f" Nenhuma coluna chave encontrada para '{nome_df}'.")
            return gpd.GeoDataFrame() if is_geo else pd.DataFrame()
            
    except Exception as e:
        warnings.warn(f" Não foi possível carregar ou processar o arquivo para '{nome_df}': {e}")
        return gpd.GeoDataFrame() if is_geo else pd.DataFrame()

# --- 2. CONFIGURAÇÃO DE CAMINHOS E CARREGAMENTO ---
BASE_DIR = Path("../data")
PATH_GEO_NORM = BASE_DIR / "geoespacial/pontos_turisticos_angola_normalizado.geojson"
PATH_CLIMA = BASE_DIR / "climatic-environmental/clima_lulc_normalizado.csv"
PATH_SOCIO = BASE_DIR / "economicsocial/economicsocial_normalizado.csv"
PATH_MOB = BASE_DIR / "climatic-environmental/mobilidade_infra_normalizado.csv"
OUTPUT_PATH = BASE_DIR / "model_inputs/model_input.csv"
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

print("\n Carregando e preparando as fontes de dados...")

chaves_geo = ["pontoturistico"]
chaves_clima = ["nomepontoturistico"]
chaves_socio = ["poinome", "nome"]
chaves_mob = ["nome", "poinome"]

gdf_norm = carregar_e_preparar(PATH_GEO_NORM, chaves_geo, "Geo Normalizado", is_geo=True)
df_clima = carregar_e_preparar(PATH_CLIMA, chaves_clima, "Clima")
df_socio = carregar_e_preparar(PATH_SOCIO, chaves_socio, "Socioeconômico")
df_mob = carregar_e_preparar(PATH_MOB, chaves_mob, "Mobilidade")

# --- 3. AGREGAÇÃO DO DATAFRAME GEOESPACIAL ---
print("\n Agregando features geoespaciais por ponto turístico...")
if not gdf_norm.empty:
    # 3.1. Criar a base com informações únicas (uma linha por ponto turístico)
    df_base = gdf_norm[['pontoturistico', 'provincia', 'poilat', 'poilon', 'key']].drop_duplicates(subset='key').reset_index(drop=True)

    # 3.2. Agregação Numérica: Calcular estatísticas das features espaciais
    features_numericas = ['distnearestfeaturem', 'featureswithin1km']
    existing_numeric = [c for c in features_numericas if c in gdf_norm.columns]
    
    if existing_numeric:
        agg_numeric = gdf_norm.groupby('key')[existing_numeric].agg(['mean', 'std', 'max']).reset_index()
        agg_numeric.columns = ['_'.join(col).strip() for col in agg_numeric.columns.values]
        agg_numeric = agg_numeric.rename(columns={'key_': 'key'})
        print(f"  Features numéricas agregadas: {agg_numeric.shape[1]-1} novas colunas.")
    else:
        agg_numeric = pd.DataFrame({'key': df_base['key']}) # Cria df vazio com a key
        print("Nenhuma feature numérica para agregar.")

    # 3.3. Agregação Categórica: Contar a ocorrência de cada tipo de feature
    features_categoricas = ['tourism', 'amenity', 'leisure', 'natural', 'landuse', 'highway']
    existing_categorical = [c for c in features_categoricas if c in gdf_norm.columns]
    
    if existing_categorical:
        dummies = pd.get_dummies(gdf_norm[['key'] + existing_categorical], columns=existing_categorical, prefix_sep='_')
        agg_categorical = dummies.groupby('key').sum().reset_index()
        print(f"Features categóricas agregadas: {agg_categorical.shape[1]-1} novas colunas.")
    else:
        agg_categorical = pd.DataFrame({'key': df_base['key']}) # Cria df vazio com a key
        print(" Nenhuma feature categórica para agregar.")

    # --- 4. JUNÇÃO (MERGE) DE TODOS OS DATASETS ---
    print("\n Juntando todos os datasets em uma única tabela...")
    
    # Começar com a base e juntar as agregações
    df_final = pd.merge(df_base, agg_numeric, on='key', how='left')
    df_final = pd.merge(df_final, agg_categorical, on='key', how='left')

    # Juntar os outros datasets (clima, socio, mob)
    if not df_clima.empty:
        df_final = pd.merge(df_final, df_clima.drop(columns=[c for c in ['provincia'] if c in df_clima.columns], errors='ignore'), on="key", how="left")
    if not df_socio.empty:
        df_final = pd.merge(df_final, df_socio.drop(columns=[c for c in ['provincia'] if c in df_socio.columns], errors='ignore'), on="key", how="left", suffixes=("", "_socio"))
    if not df_mob.empty:
        df_final = pd.merge(df_final, df_mob, on="key", how="left", suffixes=("", "_mob"))

    # --- 5. LIMPEZA FINAL ---
    print("\n Realizando limpeza final...")
    df_final = df_final.rename(columns={'pontoturistico': 'poi_nome', 'poilat': 'latitude', 'poilon': 'longitude'})
    
    # Remover colunas duplicadas de merges e a chave
    cols_to_drop = [c for c in df_final.columns if c.endswith(('_socio', '_mob'))]
    cols_to_drop += ['key']
    # Remover colunas de nome originais dos datasets mergeados
    cols_to_drop += ['nomepontoturistico', 'poinome', 'nome']
    
    df_final = df_final.drop(columns=cols_to_drop, errors='ignore')
    
    # Preencher valores nulos restantes. 0 é uma escolha simples, mas pode não ser a ideal para todas as colunas.
    df_final = df_final.fillna(0)

    # --- 6. SALVAR CSV FINAL ---
    df_final.to_csv(OUTPUT_PATH, index=False)
    print(f"\n Processo concluído! O dataset final tem {df_final.shape[0]} linhas e {df_final.shape[1]} colunas.")
    print(f"   Model Input salvo em: {OUTPUT_PATH}")
    print("\n--- Amostra do Dataset Final ---")
    display(df_final.head())
else:
    print(" Processo interrompido. Arquivo geoespacial base não pôde ser carregado.")

 Iniciando a integração final de todas as fontes de dados...

 Carregando e preparando as fontes de dados...


 Socioeconômico: Arquivo carregado. Chave criada a partir de 'nome'.
 Mobilidade: Arquivo carregado. Chave criada a partir de 'nome'.

 Agregando features geoespaciais por ponto turístico...
 Processo interrompido. Arquivo geoespacial base não pôde ser carregado.
